# Installation and Imports

In [1]:
import torch
from IPython.display import clear_output
pt_version = torch.__version__
print(pt_version)

1.10.0+cu111


In [2]:
!pip install torch-scatter -f https://pytorch-geometric.com/whl/torch-${pt_version}.html
!pip install torch-sparse -f https://pytorch-geometric.com/whl/torch-${pt_version}.html
!pip install torch-cluster -f https://pytorch-geometric.com/whl/torch-${pt_version}.html
!pip install torch-spline-conv -f https://pytorch-geometric.com/whl/torch-${pt_version}.html
!pip install torch-geometric
!pip install torch-geometric-temporal
clear_output()

> This took about 10 mins, whil above one took almost 22 mins

```
# Install required packages.
!pip install -q torch-scatter -f https://data.pyg.org/whl/torch-1.10.0+cu113.html
!pip install -q torch-sparse -f https://data.pyg.org/whl/torch-1.10.0+cu113.html
!pip install -q git+https://github.com/pyg-team/pytorch_geometric.git
!pip install torch-geometric-temporal

```

# Dataset

In [3]:
from torch_geometric_temporal.dataset import METRLADatasetLoader
loader = METRLADatasetLoader()
dataset = loader.get_dataset(num_timesteps_in=12, num_timesteps_out=12)
print("Dataset type:  ", dataset)
print("Number of samples / sequences: ",  len(set(dataset)))

Dataset type:   <torch_geometric_temporal.signal.static_graph_temporal_signal.StaticGraphTemporalSignal object at 0x7f1f5f7b9e50>
Number of samples / sequences:  34249


In [5]:
next(iter(dataset))

Data(x=[207, 2, 12], edge_index=[2, 1722], edge_attr=[1722], y=[207, 12])

- 207 nodes
- 2 features per node (speed, time)
- 12 timesteps per bucket (12 x 5 min = 60 min) 
- Labels for 12 future timesteps (normalized speed) --> node regression
- Edge_attr is build based on the distances between sensors + threshold

In [6]:
METRLADatasetLoader??

> Above Dataset is an example of Sequence to Sequnce

In [7]:
# Important: It is not always like that!
from torch_geometric_temporal.dataset import ChickenpoxDatasetLoader
d = ChickenpoxDatasetLoader().get_dataset(lags=4)
next(iter(d))

Data(x=[20, 4], edge_index=[2, 102], edge_attr=[102], y=[20])

> This one is a typical example of 'predict-next-timestamp'

In [8]:
from torch_geometric_temporal.signal import temporal_signal_split

In [10]:
train_dataset, test_dataset = temporal_signal_split(dataset, train_ratio=0.8)

In [11]:
print("Number of train buckets:", len(set(train_dataset)))
print('Number of test buckets:',len(set(test_dataset)))

Number of train buckets: 27399
Number of test buckets: 6850


# Model Part

In [13]:
import torch
import torch.nn.functional as F
from torch_geometric_temporal.nn.recurrent import A3TGCN

In [14]:
A3TGCN??

In [15]:
class TemporalGNN(torch.nn.Module):
  def __init__(self, node_features, periods):
    super(TemporalGNN, self).__init__()
    self.tgnn=A3TGCN(in_channels=node_features,
                     out_channels=32,
                     periods=periods)
    self.linear=torch.nn.Linear(32, periods)

  def forward(self, x, edge_index):
    h=self.tgnn(x, edge_index)
    h=F.relu(h)
    h=self.linear(h)
    return h

# Training

In [19]:
torch.cuda.is_available()

False

In [20]:
device=torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cpu')

In [21]:
model=TemporalGNN(node_features=2, periods=12).to(device)

In [22]:
optimizer=torch.optim.Adam(model.parameters(), lr=0.01)

In [23]:
model.train()

TemporalGNN(
  (tgnn): A3TGCN(
    (_base_tgcn): TGCN(
      (conv_z): GCNConv(2, 32)
      (linear_z): Linear(in_features=64, out_features=32, bias=True)
      (conv_r): GCNConv(2, 32)
      (linear_r): Linear(in_features=64, out_features=32, bias=True)
      (conv_h): GCNConv(2, 32)
      (linear_h): Linear(in_features=64, out_features=32, bias=True)
    )
  )
  (linear): Linear(in_features=32, out_features=12, bias=True)
)

In [24]:
max_iters=100

In [25]:
for epoch in range(10):
  loss=0
  step=0
  for snapshot in train_dataset:
    snapshot=snapshot.to(device)
    y_hat=model(snapshot.x, snapshot.edge_index)
    loss=loss + torch.mean((y_hat-snapshot.y)**2)
    step+=1
    if step > max_iters:
      break
  
  loss = loss / (step + 1)
  loss.backward()
  optimizer.step()
  optimizer.zero_grad()
  print('Epoch {} train_mse: {:.4f}'.format(epoch, loss.item()))

Epoch 0 train_mse: 0.6384
Epoch 1 train_mse: 0.6232
Epoch 2 train_mse: 0.6115
Epoch 3 train_mse: 0.5986
Epoch 4 train_mse: 0.5840
Epoch 5 train_mse: 0.5685
Epoch 6 train_mse: 0.5537
Epoch 7 train_mse: 0.5406
Epoch 8 train_mse: 0.5287
Epoch 9 train_mse: 0.5165


> Evaluation

In [27]:
model.eval()
loss=0
step=0

predictions=[]
labels=[]

for snapshot in test_dataset:
  snapshot = snapshot.to(device)
  y_hat=model(snapshot.x, snapshot.edge_index)
  loss=loss +torch.mean((y_hat - snapshot.y)**2)
  labels.append(snapshot.y)
  predictions.append(y_hat)
  step += 1
  if step > 50:
    break

loss= loss/ (step+1)
loss=loss.item()
print("Test MSE: {:.4f}".format(loss))

Test MSE: 0.5225
